In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import os
import librosa
import librosa.display
import IPython.display as ipd
from itertools import cycle
from tqdm import tqdm

In [2]:
class Configuration:

  class Paths:
    train_audio_folders = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'
    train_audio_files='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio/*/*.ogg'
    unlabeld_soundspaces='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/unlabeled_soundscapes'
    taxonomy_birds = '/content/drive/MyDrive/Kaggle/Bird_CLEF2024/eBird_Taxonomy_v2021.csv'
    meta_data='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_metadata.csv'
    sample_submission='/content/drive/MyDrive/Kaggle/Bird_CLEF2024/sample_submission.csv'


  class ClassInfo:
    class_names=sorted(os.listdir('/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio'))
    num_classes=len(class_names)
    labels=list(range(num_classes))
    labels_with_names = dict(zip(labels,class_names))

config=Configuration()

In [3]:
labels_names=config.ClassInfo.labels_with_names
meta_data=pd.read_csv(config.Paths.meta_data)
meta_data['filepath'] = config.Paths.train_audio_folders+'/'+meta_data.filename

In [24]:
df= meta_data.set_index('primary_label')
(df.loc[bird].iloc[0,:]['filepath'])

'/content/drive/MyDrive/Kaggle/Bird_CLEF2024/train_audio/asbfly/XC134896.ogg'

In [18]:
birds=meta_data['primary_label'].unique()
df= meta_data.set_index('primary_label')
bird_species=[]
audio_signal=[]
sample_rate=[]
audio_data={}
for bird in birds:
  for i in tqdm(range(df.loc[bird].shape[0]),desc=f'Loading audio signal and sample rate for {bird}'):
    path = df.loc[bird].iloc[0,:]['filepath']
    audio, sr = librosa.load(path)
    bird_species.append(bird)
    audio_signal.append(audio)
    sample_rate.append(sr)

audio_data={'bird_species':bird_species,
'audio_signal':audio_signal,
'sample_rate':sample_rate}

Loading audio signal and sample rate for blaeag1: 100%|██████████| 6/6 [00:00<00:00, 51.90it/s]
Loading audio signal and sample rate for blakit1:  96%|█████████▌| 222/231 [00:16<00:00, 13.41it/s]


KeyboardInterrupt: 

In [16]:
df.loc[bird].shape[0]

500